In [ ]:
import numpy as np
import pandas
import urllib
from bs4 import BeautifulSoup as bs
import os
import re
from datetime import datetime
import time

os.chdir('/home/idies/workspace/Storage/raddick/raddick_folksonomy/data/secondpass')
print('ok')

# Load data file

In [ ]:
# Read raw data
inmusic_df = pandas.read_csv('cleaned_audio_data.csv')
# Use music_df for processing; move rows to output when scored
music_df = inmusic_df

# IDs are unique, we can use them as pandices IWW - but now let's use rownumber #music_df.groupby('ID').size().sort_values(ascending=False)
print('Datafile read: {0:,.0f} rows.'.format(len(inmusic_df)))

# rename columns
music_df.index.name = 'rownumber'
music_df = music_df.rename(columns={'ID': 'id', 'Source': 'source', 'Sample File Name': 'filename'})
print('Columns renamed')

music_df = music_df.assign(filetype = music_df['filename'].apply(lambda x: x.split('.')[-1].strip()))
music_df = music_df.assign(stem = music_df['filename'].apply(lambda x: x[0:x.rfind('.')]))

print('backing up...')
music_df_bk = music_df

#print(len(music_df))

print('ok')
#music_df

# Make columns and fix a few values by hand

In [ ]:
print('reading from backup...')
music_df = music_df_bk
music_df = music_df.assign(date_format = 'otros')
music_df = music_df.assign(date_string = np.nan)
music_df = music_df.assign(thedate = pandas.to_datetime(np.nan))
music_df = music_df.assign(mics_etc = np.nan)

music_df = music_df.assign(theyear = np.nan)
music_df = music_df.assign(themonth = np.nan)
music_df = music_df.assign(theday = np.nan)

music_df.loc[music_df['stem'] == '101-Dyin\' to Know -Joanne Shaw Taylor-The Sage Gateshead 2017-0-11-07', 
            ['date_string', 'thedate', 'date_format']] = ['2017-0-11-07', pandas.to_datetime('2017-11-07'), 'byhand']
music_df.loc[music_df['stem'] == 'john2017-0-21d01t01', 
            ['date_string', 'theyear', 'date_format']] = ['2017-0-21', 2017, 'byhand']
music_df.loc[music_df['stem'] == 'Osvaldo Golijov Azul (2006) -cck BA 2017-0-07-01', 
            ['date_string', 'thedate', 'date_format']] = ['2017-0-07-01', pandas.to_datetime('2017-07-01'), 'byhand']
music_df.loc[music_df['stem'] == 'gd68-00-00MidnightHour', ['date_string', 'theyear', 'date_format']] = ['68-00-00', 1968, 'byhand']
music_df.loc[music_df['stem'] == 'gd66-11-00.FM.interview', ['date_string', 'theyear', 'themonth', 'date_format']] = ['66-11-00', 1968, 11, 'byhand']
music_df.loc[music_df['stem'] == 'hartford1986-10-0xCulpepperT01', ['date_string', 'theyear', 'themonth', 'date_format']] = ['1986-10-0x', 1986, 10, 'byhand']
music_df.loc[music_df['stem'] == 'Radiohead_Lallapoloza_Chicago_August1-2008_T01-1644', ['date_string', 'thedate', 'date_format']] = ['August1-2008', pandas.to_datetime('2008-08-01'), 'byhand']
music_df.loc[music_df['stem'] == 'Love & Rockets_November22-1986_T01', ['date_string', 'thedate', 'date_format']] = ['November22-1986', pandas.to_datetime('1986-11-22'), 'byhand']
music_df.loc[music_df['stem'] == 'Toronto 17April80 101', ['date_string', 'thedate', 'date_format']] = ['17April80', pandas.to_datetime('1980-04-17'), 'byhand']
music_df.loc[music_df['stem'] == 'Montreal 24April80 101', ['date_string', 'thedate', 'date_format']] = ['24April80', pandas.to_datetime('1980-04-24'), 'byhand']
music_df.loc[music_df['stem'] == 'BOB DYLAN - Lewiston August 16th 1994_Disc 01 Track 01', ['date_string', 'thedate', 'date_format']] = ['August 16th 1994', pandas.to_datetime('1994-08-16'), 'byhand']
music_df.loc[music_df['stem'] == 'Toronto19April80 101', ['date_string', 'thedate', 'date_format']] = ['19April80', pandas.to_datetime('1980-04-19'), 'byhand']
music_df.loc[music_df['stem'] == 'Lakeland 21Nov81 101', ['date_string', 'thedate', 'date_format']] = ['21Nov81', pandas.to_datetime('1981-11-21'), 'byhand']
music_df.loc[music_df['stem'] == 'Lasse Marhaug 2010 02 14', ['date_string', 'thedate', 'date_format']] = ['2010 02 14', pandas.to_datetime('2010-02-14'), 'byhand']
music_df.loc[music_df['stem'] == 'Townes 1988nov18 01 intro', ['date_string', 'thedate', 'date_format']] = ['nov18 01', pandas.to_datetime('2001-11-08'), 'byhand']
music_df.loc[music_df['stem'] == 'QOTSA7nov2017-23', ['date_string', 'thedate', 'date_format']] = ['7nov2017', pandas.to_datetime('2017-11-07'), 'byhand']
music_df.loc[music_df['stem'] == 'London Grammar - 2017 - KÃ¶ln -01', ['date_string', 'theyear', 'themonth', 'theday', 'date_format']] = ['mar - 2017', '2017', '3', np.nan, 'byhand']
music_df.loc[music_df['stem'] == 'New Order-Transmission 5 Jodrell Bank 7 Jul 1316', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['7 Jul 13', pandas.to_datetime('2013-07-13'), '16', 'byhand']
music_df.loc[music_df['stem'] == 'AC-DC_Nov_20th_1981_cd1Track01', ['date_string', 'thedate', 'date_format']] = ['Nov_20th_1981', pandas.to_datetime('1981-11-20'), 'byhand']
music_df.loc[music_df['stem'] == 'wailers73_July_18-23.Maxs1t01', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['73_July_18', pandas.to_datetime('1973-07-18'), '-23.Maxs1t01', 'byhand']
music_df.loc[music_df['stem'] == 'Roy Harper Jimmy Page St Ives Hotel Lytham St Annes Engand 1984 May 2101', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['1984 May 21', pandas.to_datetime('1984-05-21'), '01', 'byhand']
music_df.loc[music_df['stem'] == '01_soundgarden_01131990', ['date_string', 'thedate', 'date_format']] = ['01131990', pandas.to_datetime('1990-01-13'), 'byhand']
music_df.loc[music_df['stem'] == '01_FNM_01131990', ['date_string', 'thedate', 'date_format']] = ['01131990', pandas.to_datetime('1990-01-13'), 'byhand']
music_df.loc[music_df['stem'] == 'spafford171108101', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['171108101', pandas.to_datetime('2017-11-08'), '101', 'byhand']
music_df.loc[music_df['stem'] == '0801_19403210', ['date_string', 'theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = ['0801', 2008, 1, np.nan, '_19403210', 'byhand']
music_df.loc[music_df['stem'] == 'BK201171206t01', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['2011712', pandas.to_datetime('2011-07-12'), '06t01', 'byhand']
music_df.loc[music_df['stem'] == 'IK20171200t01', ['date_string', 'theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = ['20171200', 2017, 12, np.nan, 't01', 'byhand']
music_df.loc[music_df['stem'] == 'MM20171000t01', ['date_string', 'theyear', 'themonth', 'theday', 'mics_etc', 'date_format']] = ['20171000', 2017, 10, np.nan, 't01', 'byhand']
music_df.loc[music_df['stem'] == '4:49.27       51042920 B   -b-   --   ---xx   flac  0.3234  01. Announcements', ['date_format']] = ['byhand']
music_df.loc[music_df['stem'] == '1:40.28      17705900     ---   --    -xx    police-19820407-d1t01', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['19820407', pandas.to_datetime('1982-04-07'), '-d1t01', 'byhand']
music_df.loc[music_df['stem'] == '999d594bb251168f1d2e659964302dd5  2018 02 11-210445 A Trane  Big Satan', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['2018 02 11', pandas.to_datetime('2018-02-11'), '-210445', 'byhand']
music_df.loc[music_df['stem'] == 'R081113-23205316', ['date_string', 'thedate', 'mics_etc', 'date_format']] = ['081113', pandas.to_datetime('2008-11-13'), '-23205316', 'byhand']


print('backing up...')
music_df_bk = music_df



music_df.groupby('date_format').size()
#music_df['stem'].loc[4473]


# Find filenames with proper yyyy-mm-dd date format

In [ ]:
print('reading from backup...')
music_df = music_df_bk

yyyymmddfinder = re.compile('(19|20)[\d]{2}\-[\d]{2}\-[\d]{2}')

music_df.loc[
    (music_df['date_format'] == 'otros')
    & (music_df['stem'].apply(lambda x: re.search(yyyymmddfinder,x) != None))
, 'date_format'] = 'yyyy-mm-dd'

music_df.loc[
    (music_df['date_format'] == 'yyyy-mm-dd')
, 'date_string'] = music_df['stem'][(music_df['date_format'] == 'yyyy-mm-dd')
                                   ].apply(lambda x: 
                                           x[re.search(yyyymmddfinder,x).start():re.search(yyyymmddfinder,x).end()]
                                          )
music_df.loc[(music_df['date_format'] == 'yyyy-mm-dd'), 'thedate'] = pandas.to_datetime(music_df['date_string'], errors='coerce')

print('backing up...')
music_df_bk = music_df

music_df.groupby('date_format').size()

#len(music_df['thedate'].dropna())

In [ ]:
music_df[['stem', 'date_string', 'thedate']][music_df['date_format'] != 'otros'].sample(10)

# Find filenames with date format <code>[\d]{1,4}\-[\d]{1,2}\-[\d]{1,4}</code>

In [ ]:
print('reading from backup...')
music_df = music_df_bk
dashfinder = re.compile('[\d]{1,4}\-[\d]{1,2}\-[\d]{1,4}')

music_df.loc[
    (music_df['date_format'] == 'otros') & (music_df['stem'].apply(lambda x: re.search(dashfinder,x) != None))
, 'date_format'] = 'dashfinder'

music_df.loc[
    (music_df['date_format'] == 'dashfinder')
, 'date_string'] = music_df['stem'][(music_df['date_format'] == 'dashfinder')
                                   ].apply(lambda x: 
                                           x[re.search(dashfinder,x).start():re.search(dashfinder,x).end()]
                                          )


print('backing up...')
music_df_bk = music_df

print(music_df.groupby('date_format').size())

#music_df[['date_string','thedate']][music_df['date_format'] == 'dashfinder']

#music_df.loc[2298]

## Parse acutal dates from those filenames

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

dateredict = {}
dateredict['xx-xx-xxxx'] = re.compile('[\d]{2}\-[\d]{2}\-[\d]{4}')
dateredict['x-xx-xxxx'] = re.compile('[\d]{1}\-[\d]{2}\-[\d]{4}')
dateredict['xx-x-xxxx'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{4}')
dateredict['x-x-xxxx'] = re.compile('[\d]{1}\-[\d]{1}\-[\d]{4}')

dateredict['xxxx-xx-xx'] = re.compile('[\d]{4}\-[\d]{2}\-[\d]{2}')
dateredict['xxxx-x-xx'] = re.compile('[\d]{4}\-[\d]{1}\-[\d]{2}')
dateredict['xxxx-xx-x'] = re.compile('[\d]{4}\-[\d]{2}\-[\d]{1}')
dateredict['xxxx-x-x'] = re.compile('[\d]{4}\-[\d]{1}\-[\d]{1}')

dateredict['xx-xx-xx'] = re.compile('[\d]{2}\-[\d]{2}\-[\d]{2}')
dateredict['x-xx-xx'] = re.compile('[\d]{1}\-[\d]{2}\-[\d]{2}')
dateredict['xx-x-xx'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{2}')
dateredict['x-x-xx'] = re.compile('[\d]{1}\-[\d]{1}\-[\d]{2}')

dateredict['xx-x-xx'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{2}')
dateredict['xx-xx-x'] = re.compile('[\d]{2}\-[\d]{2}\-[\d]{1}')
dateredict['xx-x-x'] = re.compile('[\d]{2}\-[\d]{1}\-[\d]{1}')



dates_df = pandas.DataFrame()
dates_df = dates_df.assign(date_string = music_df['date_string'][music_df['date_format'] == 'dashfinder'])
dates_df = dates_df.assign(date_format_2 = np.nan)

for nameofthisre, thisre in dateredict.items():
    dates_df.loc[(dates_df['date_format_2'].isnull()) & (dates_df['date_string'].apply(lambda x: re.search(thisre, x) != None)), 'date_format_2'] = nameofthisre

dates_df = dates_df.assign(date_string = music_df['date_string'][music_df['date_format'] == 'dashfinder'])
dates_df = dates_df.assign(part1 = dates_df['date_string'].apply(lambda x: x.split('-')[0]))
dates_df = dates_df.assign(part2 = dates_df['date_string'].apply(lambda x: x.split('-')[1]))
dates_df = dates_df.assign(part3 = dates_df['date_string'].apply(lambda x: x.split('-')[2]))
dates_df['part1'] = dates_df['part1'].apply(lambda x: int(x))
dates_df['part2'] = dates_df['part2'].apply(lambda x: int(x))
dates_df['part3'] = dates_df['part3'].apply(lambda x: int(x))    
dates_df = dates_df.assign(year = np.nan)
dates_df = dates_df.assign(month = np.nan)
dates_df = dates_df.assign(day = np.nan)
dates_df = dates_df.assign(extra = np.nan)

# Do some of the rare ones by hand to save time    
dates_df.loc[dates_df['date_string'] == '97-3-8', ['year', 'month', 'day', 'date_format_2']] = [1997, 3, 8, 'yy-m-d']
dates_df.loc[dates_df['date_string'] == '11-5-1701', ['year', 'month', 'day', 'extra', 'date_format_2']] = [2017, 11, 5, '01', 'mm-d-yyee']
dates_df.loc[dates_df['date_string'] == '11-9-2017', ['year', 'month', 'day', 'date_format_2']] = [2017, 11, 9, 'mm-d-yyyy']
dates_df.loc[dates_df['date_string'] == '93-11-1', ['year', 'month', 'day', 'date_format_2']] = [1993, 11, 1, 'yy-mm-d']
dates_df.loc[dates_df['date_string'] == '02-05-9', ['year', 'month', 'day', 'date_format_2']] = [2002, 5, 9, 'yy-mm-d']
dates_df.loc[dates_df['date_string'] == '2005-4-4', ['year', 'month', 'day', 'date_format_2']] = [2005, 4, 4, 'yyyy-m-d']
dates_df.loc[dates_df['date_string'] == '1984-9-3', ['year', 'month', 'day', 'date_format_2']] = [1984, 9, 3, 'yyyy-m-d']
dates_df.loc[dates_df['date_string'] == '6-11-0401', ['year', 'month', 'day', 'extra', 'date_format_2']] = [2004, 6, 11, '01', 'm-d-yyee']
dates_df.loc[dates_df['date_string'] == '2-08-2014', ['year', 'month', 'day', 'date_format_2']] = [2014, 2, 8, 'm-dd-yyyy']
dates_df.loc[dates_df['date_string'] == '9-12-2007', ['year', 'month', 'day', 'date_format_2']] = [2007, 9, 12, 'm-dd-yyyy']
dates_df.loc[dates_df['date_string'] == '71-4-22', ['year', 'month', 'day', 'date_format_2']] = [1971, 4, 22, 'yy-m-dd']
dates_df.loc[dates_df['date_string'] == '11-2-17', ['year', 'month', 'day', 'date_format_2']] = [2017, 11, 2, 'mm-d-yy']
dates_df.loc[dates_df['date_string'] == '11-3-17', ['year', 'month', 'day', 'date_format_2']] = [2017, 11, 3, 'mm-d-yy']
dates_df.loc[dates_df['date_string'] == '5-3-7303', ['year', 'month', 'day', 'extra', 'date_format_2']] = [1973, 5, 3, '03', 'm-d-yyee']
dates_df.loc[dates_df['date_string'] == '8-9-1701', ['year', 'month', 'day', 'extra', 'date_format_2']] = [2017, 8, 9, '01', 'm-d-yyee']
dates_df.loc[dates_df['date_string'] == '9-9-2017', ['year', 'month', 'day', 'date_format_2']] = [2017, 9, 9, 'm-d-yyyy']
dates_df.loc[dates_df['date_string'] == '9-7-2017', ['year', 'month', 'day', 'date_format_2']] = [2017, 9, 7, 'm-d-yyyy']


dates_df.loc[dates_df['date_format_2'] == 'xxxx-xx-x', ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][dates_df['date_format_2'] == 'xxxx-xx-x'], dates_df['part2'][dates_df['date_format_2'] == 'xxxx-xx-x'], dates_df['part3'][dates_df['date_format_2'] == 'xxxx-xx-x'], 'yyyy-mm-d']
dates_df.loc[(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)].apply(lambda x: (x % 100) + 1900), dates_df['part2'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)], dates_df['part3'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)], dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 > 18)].apply(lambda x: str(x)[0:2]), 'eeyy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'xxxx-xx-xx') & dates_df['part1'].apply(lambda x: x % 100 <= 18), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))].apply(lambda x: (x % 100) + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))], dates_df['part3'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))], dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-xx-xx') & (dates_df['part1'].apply(lambda x: x % 100 <= 18))].apply(lambda x: str(x)[0:2]), 'eeyy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18)].apply(lambda x: x + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] > 18)], 'm-d-yy']
dates_df.loc[(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18)].apply(lambda x: x + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-x-xx') & (dates_df['part3'] <= 18)], 'm-d-yy']

dates_df.loc[(dates_df['date_format_2'] == 'xxxx-x-xx'), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xxxx-x-xx')], dates_df['part2'][(dates_df['date_format_2'] == 'xxxx-x-xx')], dates_df['part3'][(dates_df['date_format_2'] == 'xxxx-x-xx')], 'yyyy-m-dd']

dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)].apply(lambda x: int(np.floor(x / 100)) + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] < 1900)].apply(lambda x: '{:02}'.format(x % 100)), 'mm-dd-yyee']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018), ['year', 'month', 'day', 'extra', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)].apply(lambda x: int(np.floor(x/100)) + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part3'] > 2018)].apply(lambda x: '{:02}'.format(x % 100)), 'mm-dd-yyee']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12)], dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xxxx') & (dates_df['part1'] > 12)], 'dd-mm-yyyy']
dates_df.loc[dates_df['date_format_2'] == 'xx-xx-xxxx', ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][dates_df['date_format_2'] == 'xx-xx-xxxx'], dates_df['part1'][dates_df['date_format_2'] == 'xx-xx-xxxx'], dates_df['part2'][dates_df['date_format_2'] == 'xx-xx-xxxx'], 'mm-dd-yyyy']

dates_df.loc[(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18)].apply(lambda x: x + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] > 18)], 'm-dd-yy']
dates_df.loc[(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18)].apply(lambda x: x + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18)], dates_df['part2'][(dates_df['date_format_2'] == 'x-xx-xx') & (dates_df['part3'] <= 18)], 'm-dd-yy']

dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31)].apply(lambda x: x + 1900), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 31)], 'yy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31),  ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31)].apply(lambda x: x + 1900), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 31)], 'mm-dd-yy']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18)].apply(lambda x: x + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18)], dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part1'] > 18)], 'dd-mm-yy']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18)].apply(lambda x: x + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18)], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part3'] > 18)], 'yy-mm-dd']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12)].apply(lambda x: x + 2000), dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12)], dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx') & (dates_df['part2'] > 12)], 'mm-dd-yy']
dates_df.loc[(dates_df['date_format_2'] == 'xx-xx-xx'), ['year', 'month', 'day', 'date_format_2']] = [dates_df['part1'][(dates_df['date_format_2'] == 'xx-xx-xx')].apply(lambda x: x + 2000), dates_df['part2'][(dates_df['date_format_2'] == 'xx-xx-xx')], dates_df['part3'][(dates_df['date_format_2'] == 'xx-xx-xx')], 'yy-mm-dd']

#print(dates_df.groupby('date_format_2').size().sort_values(ascending=False))

dates_df = dates_df.assign(thedate=pandas.to_datetime(dates_df[['year', 'month', 'day']], errors='coerce'))

music_df.loc[dates_df.index, 'thedate'] = dates_df['thedate']
music_df.loc[dates_df.index, 'date_format'] = dates_df['date_format_2']

print('backing up...')
music_df_bk = music_df
#music_df[['date_string', 'thedate', 'date_format']].dropna().sample(20)
music_df.groupby('date_format').size()


In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

stems_for_dates_s = music_df['stem'][
    (music_df['stem'].notnull())
    & (music_df['date_format'] == 'otros')
].dropna()

new_date_formats_df = pandas.DataFrame(data=None, columns=['stem', 'date_string', 'extras', 'thedate', 'year', 'month', 'day', 'date_format'], index=stems_for_dates_s.index)

new_date_formats_df.loc[stems_for_dates_s.index, 'stem'] = stems_for_dates_s

for x in ['year', 'month', 'day']:
    new_date_formats_df[x] = pandas.to_numeric(new_date_formats_df[x], downcast='integer', errors='coerce')
new_date_formats_df['thedate'] = pandas.to_datetime(new_date_formats_df['thedate'], errors='coerce')

#new_date_formats_df

redict = {}
redict['monthname_first'] = re.compile('([j|J]an[uary]*|[f|F]eb[ruary]*|[m|M]ar[ch]*|[a|A]pr[il]*|[m|M]ay|[j|J]un[e]*|[j|J]ul[y]*|[a|A]ug[ust]*|[s|S]ep[t]*[ember]*|[o|O]ct[ober]*|[n|N]ov[ember]*|[d|D]ec[ember]*)[_/\,\-\s]+[\d]*(st|nd|rd|th)*[_/\,\-\s]+[\d]{4}')
redict['monthname_in_middle'] = re.compile('[\d]+[_/\,\-\s]+([j|J]an[uary]*|[f|F]eb[ruary]*|[m|M]ar[ch]*|[a|A]pr[il]*|[m|M]ay|[j|J]un[e]*|[j|J]ul[y]*|[a|A]ug[ust]*|[s|S]ep[t]*[ember]*|[o|O]ct[ober]*|[n|N]ov[ember]*|[d|D]ec[ember]*)[_/\,\-\s]+[\d]+')
redict['eightstraight'] = re.compile('[\d]{8}')
redict['sixstraight'] = re.compile('[\d]{6}')
redict['other_numeric'] = re.compile('[\d]{1,4}[,_\.\-\s]+[\d]{1,4}[,_\.\-\s]+[\d]{1,4}')
redict['just_year_month'] = re.compile('[\d]{2,4}[,_\.\-\s]+[\d]{1,2}')
redict['just_year'] = re.compile('(19|20)[\d]{2}')


for nameofthisre, thisre in redict.items():
    new_date_formats_df.loc[
        (new_date_formats_df['date_format'].isnull()) 
        & (new_date_formats_df['stem'].apply(lambda x: re.search(thisre, x) != None))
        , 'date_string'] = new_date_formats_df['stem'][
            (new_date_formats_df['date_format'].isnull())
            & (new_date_formats_df['stem'].apply(lambda x: re.search(thisre, x) != None)
              )
        ].apply(lambda x: x[re.search(thisre, x).start():re.search(thisre, x).end()])
    
    new_date_formats_df.loc[
        (new_date_formats_df['date_format'].isnull())
        & (new_date_formats_df['date_string'].notnull())
        , 'date_format'] = nameofthisre

    
print('backing up new_date_formats_df\n')
new_date_formats_df_bk = new_date_formats_df

print(new_date_formats_df.groupby('date_format').size())

#new_date_formats_df['date_string'][new_date_formats_df['date_format'] == 'other_numeric'].sort_values()
#new_date_formats_df['stem'][new_date_formats_df['date_format'].isnull()].drop_duplicates().sort_values()
#new_date_formats_df['stem'].loc[4421]

#music_df.groupby('date_format').size()

#new_date_formats_df.dtypes


## Convert those date strings into real dates

In [ ]:
print('retrieving from backup...')
new_date_formats_df = new_date_formats_df_bk

seps = re.compile('[,_\.\-\s]+')
fournumbers = re.compile('[\d]{4}')

for k, v in redict.items():
    if (k in ['other_numeric']):
        pass
    elif (k in ['eightstraight']):
        print(k)
        d_s, m_s = get_dates_eightstraight(
            new_date_formats_df['date_string'][
                (new_date_formats_df['date_format'] == k)
            ]
        )
#        print(d_s.dropna().drop_duplicates().sort_values())           
            

    elif (k in ['sixstraight']):
        print(k)
#        new_date_formats_df.loc[
#            new_date_formats_df['date_string'].notnull()
#        , 'thedate'] = pandas.to_datetime(
#            new_date_formats_df['date_string'][
#                (new_date_formats_df['date_string'].notnull())
#            ], errors='coerce')
#        print(
#            new_date_formats_df[['date_string', 'thedate']][
#                (new_date_formats_df['date_format'] == k)
#                & (new_date_formats_df['thedate'].isnull())
#            ]
#        )

    elif (k in ['just_year_month', 'just_year']):
        pass
    else:
        d_s, m_s = get_dates_monthname(
            new_date_formats_df['date_string'][
                (new_date_formats_df['date_format'] == k)]
        )
        
        new_date_formats_df.loc[d_s.index, 'thedate'] = d_s
        new_date_formats_df.loc[m_s.index, 'mics_etc'] = m_s

#        print(new_date_formats_df['thedate'].loc[m_s.index])
#        print(new_date_formats_df['mics_etc'].loc[m_s.index])
#        print(new_date_formats_df[['date_string', 'thedate']].loc[z[z.isnull()].index])
#        print(new_date_formats_df['thedate']
#              [new_date_formats_df['date_format'] == k]
#             )

#new_date_formats_df[['date_string', 'thedate', 'extras']][new_date_formats_df['date_format'] == 'eightstraight'].dropna()


#new_date_formats_df[new_date_formats_df['date_string'] == '01131990']


#22121988 2019-12-22     88
#5217         12022017 2020-12-02     17
#12784        17102201 2022-10-17     01
#12964        17102601 2026-10-17     01
#5406         51042920 2051-04-29     20

#new_date_formats_df['stem'][new_date_formats_df['thedate'] == '1711-08-10']



In [ ]:
def get_dates_eightstraight(date_strings_s):
    dates_s = pandas.Series(data=pandas.to_datetime(np.nan), index=date_strings_s.index)
    mics_etc_s = pandas.Series(index=date_strings_s.index)

    dates_s = pandas.to_datetime(date_strings_s, errors='coerce')

    date_strings_to_parse_df = pandas.DataFrame(
        data = date_strings_s[(date_strings_s.apply(lambda x: len(x) == 8))].loc[dates_s.isnull()], 
        columns = ['date_string'],
        index = date_strings_s[(date_strings_s.apply(lambda x: len(x) == 8))].loc[dates_s.isnull()].index
    )
    
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        zero_and_one = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[0:2]))
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        two_three = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[2:4]))
#    date_strings_to_parse_df = date_strings_to_parse_df.assign(
#        two = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[2:3]))
#    date_strings_to_parse_df = date_strings_to_parse_df.assign(
#        three = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[3:4]))
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        four_to_eight = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[4:]))
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        four_five = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[4:6]))
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        six_eight = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[6:]))
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        zero_to_three = date_strings_to_parse_df['date_string'].apply(lambda x: str(x)[0:4]))
    
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        year = np.nan)
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        month = np.nan)
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        day = np.nan)
    date_strings_to_parse_df = date_strings_to_parse_df.assign(
        extras = np.nan)    
    
    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
        , 'year'] = date_strings_to_parse_df['zero_and_one'][
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
    ].apply(lambda x: int(x) + 1900)
    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
        , 'month'] = date_strings_to_parse_df['two_three'][
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
    ].apply(lambda x: int(x))
    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
        , 'day'] = date_strings_to_parse_df['four_five'][
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
    ].apply(lambda x: int(x))    
    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
        , 'extras'] = date_strings_to_parse_df['six_eight'][
        (date_strings_to_parse_df['zero_and_one'].apply(lambda x: int(x) > 31))
        & (date_strings_to_parse_df['two_three'].apply(lambda x: int(x) <= 12))
    ]

    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['year'].isnull())
        &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) >= 1900))
        &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) <= 2018))
        , 'year'] = date_strings_to_parse_df['four_to_eight'][
            (date_strings_to_parse_df['year'].isnull())
            &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) >= 1900))
            &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) <= 2018))
    ].apply(lambda x: int(x))

    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) >= 1900))
        &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) <= 2018))       
    , 'month'] = date_strings_to_parse_df['two_three'][
        (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) >= 1900))
        &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) <= 2018))    
    ].apply(lambda x: int(x))
    date_strings_to_parse_df.loc[
        (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) >= 1900))
        &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) <= 2018))       
    , 'day'] = date_strings_to_parse_df['zero_and_one'][
        (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) >= 1900))
        &  (date_strings_to_parse_df['four_to_eight'].apply(lambda x: int(x) <= 2018))    
    ].apply(lambda x: int(x))    

    
    print('Dates parsed so far:')
    print(date_strings_to_parse_df[['date_string', 'year', 'month', 'day', 'extras']].dropna())
    print('\n')
    print('Yet to be parsed:')
    print(date_strings_to_parse_df[date_strings_to_parse_df['year'].isnull()])
    
    return dates_s, mics_etc_s

def get_dates_monthname(date_strings_s):
    
    fourdigits = re.compile('[\d]{4}')
    
    dates_s = pandas.Series(data=pandas.to_datetime(np.nan), index=date_strings_s.index)
    mics_etc_s = pandas.Series(index=date_strings_s.index)
    
    dates_s = pandas.to_datetime(date_strings_s, errors='coerce')
    # adjust for dates that look like 4 digits but actually are not - they have 2 digits of extras

    mics_etc_s = date_strings_s[
        (~date_strings_s.index.isin(dates_s.dropna().index))
    ].apply(lambda x: x[-2:])
    
    dates_s = pandas.concat([dates_s.dropna(), 
                             pandas.to_datetime(
                                 date_strings_s[(~date_strings_s.index.isin(dates_s.dropna().index))].apply(lambda x: x[0:-2])
                                 , errors='coerce')
                            ])
    mics_etc_s = pandas.concat([mics_etc_s, 
        date_strings_s[
            dates_s.apply(lambda x: x.year < 1900)
        ].apply(lambda x: x[-2:])
                               ])
    dates_s = pandas.concat([dates_s, 
        pandas.to_datetime(date_strings_s[
            dates_s.apply(lambda x: x.year < 1900)
        ].apply(lambda x: x[0:-2])
                           , errors='coerce')
                            ])
    return dates_s, mics_etc_s

print('ok')



In [ ]:
music_df['stem'].loc[[1239, 1728, 8524, 8740, 8978, 9373, 9374, 9919, 9975, 10460, 10529, 12665, 12784, 12964, 17992, 18055, 19227]]

music_df['stem'].loc[10529]

## Merge parsed dates back into main music dataframe

In [ ]:
music_df.loc[
    (music_df['date_format'] == 'otros')
    & (music_df.index.isin(new_date_formats_df.index))
    , 'date_string'] = new_date_formats_df['date_string']

music_df.loc[
    (music_df['date_format'] == 'otros')
    #& music_df['date_string'].notnull()
    & (music_df.index.isin(new_date_formats_df.index))
, 'date_format'] = new_date_formats_df['date_format']


music_df[(music_df['date_format'] != 'otros')].groupby('date_format').size()

In [ ]:
seps = re.compile('[,_\.\-\s]+')
ordinalios = re.compile('(st|nd|rd|th)*[\,]*[\s]+')

redict = {}
redict['longmonthname_day_year'] = re.compile('([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|J]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)[,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{4}')

print('retrieving from backup...')
music_df = music_df_bk

for nameofthisre, thisre in redict.items():
    print(nameofthisre)
    #print(music_df['stem'][(music_df['date_format'] == 'otros')])
    music_df.loc[(music_df['date_format'] == 'otros') & (music_df['stem'].apply(lambda x: re.search(thisre, x) != None))
                 , 'date_string'] = music_df['stem'][
        (music_df['date_format'] == 'otros') 
        & (music_df['stem'].apply(lambda x: re.search(thisre, x) != None))
        ].apply(lambda x: x[re.search(thisre, x).start():re.search(thisre,x).end()])

#    print(
#        music_df['date_string'][
#            (music_df['date_string'].notnull()) & 
#            (music_df['date_string'].apply(lambda x: re.search(thisre, str(x)) != None))
#        ])

music_df['date_string'][(music_df['date_format'] == 'otros') & (music_df['date_string'].notnull())]

## Get year/month/day individual values from full dates

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk


music_df.loc[(music_df['thedate'].notnull()), 'theyear'] = music_df['thedate'][(music_df['thedate'].notnull())].apply(lambda x: x.year)
music_df.loc[(music_df['thedate'].notnull()), 'themonth'] = music_df['thedate'][(music_df['thedate'].notnull())].apply(lambda x: x.month)
music_df.loc[(music_df['thedate'].notnull()), 'theday'] = music_df['thedate'][(music_df['thedate'].notnull())].apply(lambda x: x.day)

print('backing up...')
music_df_bk = music_df

music_df.sample(10)

In [ ]:
print('retrieving from backup...')
music_df = music_df_bk

seps = re.compile('[,_\.\-\s]+')
ordinalios = re.compile('(st|nd|rd|th)*[\,]*[\s]+')

redict = {}
redict['longmonthname_day_year'] = re.compile('([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|J]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)[,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{2,4}')
redict['shortmonthname_day_year'] = re.compile('([j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[_,\.\-\s]*[\d]+[st|nd|rd|th]*[,\.\-\s]*[\d]{2,4}')
redict['day_longmonthname_year'] = re.compile('[\d]+(st|nd|rd|th)*[,\.\-\s]*([j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|j]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)(?![a-z])[,\.\-\s]*[\d]{2,4}')
redict['day_shortmonthname_year'] = re.compile('[\d]+(st|nd|rd|th)*[,\.\-\s]*([j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[_,\.\-\s]*[\d]{2,4}')
redict['year_longmonthname_day'] = re.compile('([\d]+[,\.\-\s]*[j|J]anuary|[f|F]ebruary|[m|M]arch|[a|A]pril|[m|M]ay|[j|j]une|[j|J]uly|[a|A]ugust|[s|S]eptember|[o|O]ctober|[n|N]ovember|[d|D]ecember)(?![a-z])[,\.\-\s]*[\d]+[st|nd|rd|th]*')
redict['year_shortmonthname_day'] = re.compile('([\d]+[,\.\-\s]*[j|J]an|[f|F]eb|[m|M]ar|[a|A]pr|[m|M]ay|[j|j]un|[j|J]ul|[a|A]ug|[s|S]ep|[o|O]ct|[n|N]ov|[d|D]ec)(?![a-z])[,\.\-\s]*[\d]+[st|nd|rd|th]*')
redict['eightstraight'] = re.compile('[\d]{8}')
redict['sixstraight'] = re.compile('[\d]{6}')
redict['other_numeric'] = re.compile('[\d]{1,4}[,_\.\-\s]+[\d]{1,4}[,_\.\-\s]+[\d]{1,4}')
redict['just_year_month'] = re.compile('(19|20)[\d]{2}[,_\.\-\s]+[\d]{1,2}')
redict['just_year'] = re.compile('(19|20)[\d]{2}')

keys_to_remove = []

for nameofthisre, thisre in redict.items():
    music_df.loc[
        (music_df['date_format'] == 'otros')
        & (music_df['stem'].apply(lambda x: re.search(thisre,x) != None))
    , 'date_format'] = nameofthisre
    music_df.loc[
        (music_df['date_format'] == nameofthisre)
    , 'date_string'] = music_df['stem'][
        (music_df['date_format'] == nameofthisre)
    ].apply(lambda x: x[re.search(thisre,x).start():re.search(thisre,x).end()])
    if (len(music_df[music_df['date_format'] == nameofthisre]) > 0):
        print('Regex {0:}: found {1:,.0f} matching filenames...'.format(nameofthisre, len(music_df[music_df['date_format'] == nameofthisre])))
    else:
        keys_to_remove.append(nameofthisre)
        
for x in keys_to_remove:
    del redict[x]

#print('backing up...')
#music_df_bk = music_df

music_df.groupby('date_format').size()

#music_df[music_df['date_string'] == '000225']



## Turn date strings found above into real dates

In [ ]:
for x in redict.keys():
    print(x)
    
music_df[['date_string', 'thedate']][music_df['date_format'] == 'longmonthname_day_year']